### Code definitions

In [1]:
import os
import sys
from pathlib import Path
from asgiref.sync import sync_to_async
import string
import pandas as  pd
from datetime import date
import yfinance as yf
import sqlite3

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'share_dinkum_proj.settings')

import django
django.setup()

from django.conf import settings
import share_dinkum_app.models as app_models

from share_dinkum_app import loading

import logging
logger = logging.getLogger(__name__)


import_data_folder = settings.BASE_DIR / 'share_dinkum_app' / 'import_data'

personal_data = import_data_folder / 'data_import_template_personal.xlsx'
sample_data = import_data_folder / 'data_import_template_public.xlsx'


defaults = {
    'sample_data' : {
        'default_name' : 'sample_data',
        'default_email' : 'sample_data@example.com',
        'password' : 'password',
        'default_portfolio_description' : 'Sample Data Portfolio',
        'input_file' : sample_data,
    },
    'own_data' : {
        'default_name' : 'admin',
        'default_email' : 'admin@example.com',
        'password' : 'password',
        'default_portfolio_description' : 'Default Portfolio',
        'input_file' : personal_data,
    },
}


@sync_to_async
def add_account(selection='sample_data'):

    default_name = defaults[selection]['default_name']
    default_email = defaults[selection]['default_email']
    password = defaults[selection]['password']
    default_portfolio_description = defaults[selection]['default_portfolio_description']

    
    fiscal_year_type, created = app_models.FiscalYearType.objects.get_or_create(
        description='Australian Tax Year',
        defaults={'start_month': 7, 'start_day': 1}
    )

    if not app_models.AppUser.objects.filter(username=default_name).exists():
        user = app_models.AppUser.objects.create_superuser(username=default_name, email=default_email, password=password)   # Please change this password after logging in
        logger.info("Superuser created successfully!")
    owner = app_models.AppUser.objects.get(username=default_name)

    record = {'description': default_portfolio_description, 'owner': owner, 'fiscal_year_type': fiscal_year_type}
    account, created = app_models.Account.objects.get_or_create(description=default_portfolio_description, defaults=record)
    return account

@sync_to_async
def clear_all_data():
    loading.DataLoader.clear_all_data()
    try:
        del account # To avoid stale references
    except NameError:
        pass

@sync_to_async
def load_all_data(account, input_file):
    loading.DataLoader(account=account, input_file=input_file)

@sync_to_async
def load_historical_prices(df, account):
    loader = loading.DataLoader(account=account, input_file=None)
    loader.load_table_to_model(model=app_models.InstrumentPriceHistory, df=df)
    
@sync_to_async
def update_all_history(account):
    account.update_all_price_history()
    account.update_all_exchange_rate_history()
    account.save()

@sync_to_async
def backup(name='main'):
    backup_path = Path('backups') / name
    loading.DataBackupManager.backup(backup_path)

@sync_to_async
def restore(name='main'):
    backup_path = Path('backups') / name
    loading.DataBackupManager.restore(backup_path)


def get_selection():

    selection = 'sample_data'

    if defaults['own_data']['input_file'].exists():
        res = input("Personal import file exists. Do you want to use it? (y/n): ")
        if res.upper() == 'Y':
            selection = 'own_data'
            logger.info('Using own data')
        else:
            selection = 'sample_data'
            logger.info('Using generic data')
        
        return selection


### Clear current data

In [2]:
await clear_all_data()

2025-08-30 15:09:15 [INFO] share_dinkum_app.loading: Deleted all models
2025-08-30 15:09:15 [INFO] share_dinkum_app.loading: Forcefully deleted and recreated folder: C:\code\share-dinkum\share_dinkum_proj\media


### Load data

In [3]:
selection = get_selection()
input_file = defaults[selection]['input_file']
account = await add_account(selection=selection)
await load_all_data(account=account, input_file=input_file)

2025-08-30 15:09:19 [INFO] __main__: Using generic data
2025-08-30 15:09:20 [INFO] __main__: Superuser created successfully!
2025-08-30 15:09:20 [INFO] share_dinkum_app.loading: Loading Market


100%|██████████| 11/11 [00:00<00:00, 166.84it/s]

2025-08-30 15:09:20 [INFO] share_dinkum_app.loading: Loading Instrument



100%|██████████| 2349/2349 [00:25<00:00, 91.18it/s]

2025-08-30 15:09:47 [INFO] share_dinkum_app.loading: Loading Buy
2025-08-30 15:09:47 [ERROR] share_dinkum_app.utils.filefield_operations: File C:\full_path_to_file\file.pdf does not exist.
2025-08-30 15:09:47 [ERROR] share_dinkum_app.utils.filefield_operations: File C:\full_path_to_file\file.pdf does not exist.


...


100%|██████████| 15/15 [00:03<00:00,  4.13it/s]

2025-08-30 15:09:54 [INFO] share_dinkum_app.loading: Loading Dividend
2025-08-30 15:09:54 [ERROR] share_dinkum_app.utils.filefield_operations: File C:\full_path_to_file\file.pdf does not exist.
2025-08-30 15:09:54 [ERROR] share_dinkum_app.utils.filefield_operations: File C:\full_path_to_file\file.pdf does not exist.



100%|██████████| 8/8 [00:00<00:00, 63.22it/s]

2025-08-30 15:09:54 [INFO] share_dinkum_app.loading: Loading Distribution
2025-08-30 15:09:54 [ERROR] share_dinkum_app.utils.filefield_operations: File C:\full_path_to_file\file.pdf does not exist.
2025-08-30 15:09:54 [ERROR] share_dinkum_app.utils.filefield_operations: File C:\full_path_to_file\file.pdf does not exist.



100%|██████████| 58/58 [00:00<00:00, 90.16it/s]


### Load historical data from a different database

In [ ]:
# You can ignore this cell

legacy_db = Path(r'C:\code\finance-database\finance-database.db')

if selection == 'own_data' and legacy_db.exists():
    query = "SELECT * FROM [price_history] where ticker_code != 'AUDUSD=X'"
    parameters = None
    with sqlite3.connect(legacy_db) as conn:
        df = pd.read_sql_query(query, conn, params=parameters)
    df['account'] = account
    df['instrument__name'] = df['ticker_code'].apply(lambda x : x.split('.')[0])
    df['date'] = df['date'].apply(lambda x : date.fromisoformat(x))
    df = df.drop(columns=['id', 'ticker_code', 'capital_gains', 'dividends'])
    df = df[~df['high'].isna()]

    await load_historical_prices(df=df, account=account)

### Update price history

In [4]:
await update_all_history(account=account)

2025-08-30 15:10:03 [INFO] share_dinkum_app.yfinanceinterface: Fetching price history for VGS.AX
2025-08-30 15:10:04 [INFO] share_dinkum_app.yfinanceinterface: Fetching price history for AFI.AX


### Data backup

In [ ]:
await backup(name='main')

2025-08-30 15:10:16 [INFO] share_dinkum_app.loading: Creating DataExport for 1 accounts
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals: Starting data export process.
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals:     - AppUser
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals:     - FiscalYearType
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals:     - FiscalYear
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals:     - Account
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals:     - LogEntry
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals:     - ExchangeRate
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals:     - Market
2025-08-30 15:10:16 [INFO] share_dinkum_app.signals:     - Instrument
2025-08-30 15:10:17 [INFO] share_dinkum_app.signals:     - InstrumentPriceHistory
2025-08-30 15:10:17 [INFO] share_dinkum_app.signals:     - Buy
2025-08-30 15:10:18 [INFO] share_dinkum_app.signals:     - Sell
2025-08-30 15:10:18 [INFO] share_dinkum_app.signals:     - Parcel


#### Data Restore

In [ ]:
# This overwrites existing data!
# await restore(name='main')

2025-08-30 15:10:50 [INFO] share_dinkum_app.loading: Restoring SQLite DB from backups\main\db.sqlite3 to db.sqlite3
2025-08-30 15:10:50 [INFO] share_dinkum_app.loading: Restore completed successfully.
